In [ ]:
# Fazendo o download do arquivo em json da empresa Loggi.
# Aonde se quer achar os locais de hub (saída de entrega) e locais de deliveries(destinatário) por onde a empresa faz entregas no estado do DF.

!wget -q << EOF https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json EOF \-O deliveries.json

In [ ]:
# Amostra de como é o arquivo original antes de manipula-lo

with open(file='deliveries.json', mode='r', encoding='utf8')as file:
  data=json.load(file)

deliveries_df = pd.DataFrame(data)
deliveries_df.head()

In [ ]:
# Importação de bibliotecas e pacotes.
# Ajustando as colunas (origin) e (deliveries) para que saissem de uma lista dentro do DataFrame.

import json
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas

with open(file='deliveries.json', mode='r', encoding='utf8')as file:
  data=json.load(file)

deliveries_df = pd.DataFrame(data)

# Coluna origin:
hub_origin_df = pd.json_normalize(deliveries_df['origin'])

deliveries_df= pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)
deliveries_df = deliveries_df.drop('origin', axis=1)
deliveries_df = deliveries_df[['name', 'region', 'lng', 'lat', 'vehicle_capacity', 'deliveries']]
deliveries_df.rename(columns={'lng':'hub_lng', 'lat':'hub_lat'}, inplace=True)

# Coluna deliveries
deliveries_explod_df = deliveries_df[['deliveries']].explode('deliveries')

deliveries_normalized_df = pd.concat([
    pd.DataFrame(deliveries_explod_df['deliveries'].apply(lambda record: record['size'])).rename(columns={'deliveries': 'delivery_size'}),
    pd.DataFrame(deliveries_explod_df['deliveries'].apply(lambda record: record['point']['lng'])).rename(columns={'deliveries': 'delivery_lng'}),
    pd.DataFrame(deliveries_explod_df['deliveries'].apply(lambda record: record['point']['lat'])).rename(columns={'deliveries': 'delivery_lat'}),
], axis=1)

deliveries_df = deliveries_df.drop('deliveries', axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)
deliveries_df.head()

In [ ]:
# Neste código iremos usar algumas bibliotecas para retornar algumas localizaçoes.
# Estas localizaçoes serão baseadas pela lng e lat que nos pegamos no código anterior.
# Com ela saberemos de onde sai as mercadorias e aonde elas serão entregues.
# Fazendo isso será possivel gerar gráficos que nos mostrarão exatamente o mapa de entregas da Loggi no DF.

hub_df = deliveries_df[['region', 'hub_lng', 'hub_lat']]
hub_df = hub_df.drop_duplicates().sort_values(by='region').reset_index(drop=True)

geolocator = Nominatim(user_agent='ebac_geocoder')
location = geolocator.reverse('-15.80511751066334, -47.89366206897872')

geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

hub_df['coordinates'] = hub_df['hub_lat'].astype(str) + ', ' + hub_df['hub_lng'].astype(str)
hub_df['geodata'] = hub_df['coordinates'].apply(geocoder)

hub_geodata_df = pd.json_normalize(hub_df['geodata'].apply(lambda data: data.raw))

hub_geodata_df = hub_geodata_df[['address.town', 'address.suburb', 'address.city']]
hub_geodata_df.rename(columns={'address.town':'hub_town', 'address.suburb':'hub_suburb', 'address.city':'hub_city'}, inplace=True)
hub_geodata_df['hub_city'] = np.where(hub_geodata_df['hub_city'].notna(), hub_geodata_df['hub_city'], hub_geodata_df['hub_town'])
hub_geodata_df['hub_suburb'] = np.where(hub_geodata_df['hub_suburb'].notna(), hub_geodata_df['hub_suburb'], hub_geodata_df['hub_city'])
hub_geodata_df = hub_geodata_df.drop('hub_town', axis=1)
hub_geodata_df.head()

hub_df = pd.merge(left= hub_df, right= hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[['region', 'hub_suburb', 'hub_city']]

deliveries_df = pd.merge(left= deliveries_df, right= hub_df, how= 'inner', on= 'region')
deliveries_df = deliveries_df[['name', 'region', 'hub_lng', 'hub_lat', 'hub_city', 'hub_suburb', 'vehicle_capacity', 'delivery_size', 'delivery_lng', 'delivery_lat']]

# Aqui iremos fazer o download dos deliveries que teve que ser feito por outra ferramenta na própria maquina.
# Pelo faot dos delveries serem muito grandes, levaria cerca de 7 dias para se rodar ele por completo no colab.
!wget -q << EOF https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv EOF \-O deliveries-geodata.csv

deliveries_geodata_df = pd.read_csv('deliveries-geodata.csv')

deliveries_df = pd.merge(left=deliveries_df, right=deliveries_geodata_df[["delivery_city", "delivery_suburb"]], how="inner", left_index=True, right_index=True)
deliveries_df.head()

In [ ]:
# Neste código iremos gerar dois gráficos: Barras e um aonde nos mostrará os locais e aréas de distribuição da Loggi.
# Também conseguiremos ver mais detalhadamente aonde é mais fácil de se entregar, aonde é mais difícil, qual a maior e a menor área e etc.

# Aqui estamos fazendo um download das áread do DF edescompactando-as para se poder usar no restante do código.
!wget -q << EOF https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip EOF \-O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps
!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx

mapa = geopandas.read_file("distrito-federal.shp")
mapa = mapa.loc[[0]]

hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]].drop_duplicates( ).reset_index(drop=True)
geo_hub_df = geopandas.GeoDataFrame( hub_df, geometry=geopandas.points_from_xy( hub_df["hub_lng"], hub_df["hub_lat"]))

geo_deliveries_df = geopandas.GeoDataFrame( deliveries_df,geometry=geopandas.points_from_xy( deliveries_df["delivery_lng"],deliveries_df["delivery_lat"]))

# cria o plot vazio
fig, ax = plt.subplots(figsize = (50/2.54, 50/2.54))

# plot mapa do distrito federal
mapa.plot(ax=ax, alpha=0.4, color="lightgrey")

# plot das entregas
geo_deliveries_df.query("region == 'df-0'").plot( ax=ax, markersize=1, color="red", label="df-0" )
geo_deliveries_df.query("region == 'df-1'").plot( ax=ax, markersize=1, color="blue", label="df-1" )
geo_deliveries_df.query("region == 'df-2'").plot( ax=ax, markersize=1, color="seagreen", label="df-2" )

# plot dos hubs
geo_hub_df.plot( ax=ax, markersize=30, marker="x", color="black", label="hub" )

# plot da legenda
plt.title( "Entregas no Distrito Federal por Região", fontdict={"fontsize": 16} )
lgnd = plt.legend(prop={"size": 15})
for handle in lgnd.legendHandles:
  handle.set_sizes([50])

  print('\nNeste mapa vemos que as regiões do df-0 e df-2 contem a maior área dispersa de entregas,\nem comparação com a df-1 que é mais concentrada, elas são bem mais afastadas.\n')

data = pd.DataFrame(deliveries_df[['region',  'vehicle_capacity']].value_counts(normalize=True)).reset_index()
data.rename(columns={0: "region_percent"}, inplace=True)

# Gráfivo de barras:
with sns.axes_style('whitegrid'):
  grafico = sns.barplot( data=data, x="region", y="region_percent", ci=None, palette="pastel" )
  grafico.set( title='Proporção de entregas por região', xlabel='Região', ylabel='Proporção' )

  print('\nCom o gráfico de barras abaixo, vemos perfeitamente que a região com o maior índice de entregas é de fato o df-1\n')